In [1]:
# --- PASSO 1: PREPARAÇÃO E CONEXÃO DO PIPELINE ---

# Importando as bibliotecas essenciais para o pipeline
import pandas as pd
import gspread

print("--- Iniciando Pipeline de Automação de Análise de Risco ---")
print("Passo 1: Conectando à fonte de dados no Google Sheets...")

# Bloco de Autenticação
try:
    gc = gspread.service_account(filename=r'meus-projetos-de-dados-7a664330fb60.json')
    planilha = gc.open("Base_de_Dados_RH_Automação") 
    aba_dados_brutos = planilha.worksheet("dados_brutos")
    df = pd.DataFrame(aba_dados_brutos.get_all_records())

    print("\n✅ Conexão estabelecida e dados carregados com sucesso!")
    print(f"Foram encontradas {df.shape[0]} linhas na planilha.")
    
except Exception as e:
    print(f"\n❌ Ocorreu um erro na conexão: {e}")

--- Iniciando Pipeline de Automação de Análise de Risco ---
Passo 1: Conectando à fonte de dados no Google Sheets...

✅ Conexão estabelecida e dados carregados com sucesso!
Foram encontradas 1473 linhas na planilha.


In [5]:
# --- PASSO 2: LIMPEZA E PREPARAÇÃO DOS DADOS ---

print("\nPasso 2: Aplicando limpeza e traduções...")

try:
    # Criando uma cópia para evitar avisos de 'SettingWithCopyWarning'
    df_limpo = df.copy()

    # Remoção de colunas que não usaremos na análise de risco
    colunas_para_remover = ['EmployeeCount','Over18', 'StandardHours']
    df_limpo.drop(columns=colunas_para_remover, inplace=True)

    # Dicionários para tradução
    dicionario_saida = {'No': 'Ficou', 'Yes': 'Saiu'}
    dicionario_overtime = {'No': 'Não', 'Yes': 'Sim'}
    dicionario_cargos = {
        'Sales Executive': 'Executivo(a) de Vendas',
        'Research Scientist': 'Cientista de Pesquisa',
        'Laboratory Technician': 'Técnico(a) de Laboratório',
        'Manufacturing Director': 'Diretor(a) de Manufatura',
        'Healthcare Representative': 'Representante de Saúde',
        'Manager': 'Gerente',
        'Sales Representative': 'Representante de Vendas',
        'Research Director': 'Diretor(a) de Pesquisa',
        'Human Resources': 'Recursos Humanos'
    }

    # Aplicando as traduções para criar novas colunas
    df_limpo['Situacao'] = df_limpo['Attrition'].map(dicionario_saida)
    df_limpo['HorasExtras_PT'] = df_limpo['OverTime'].map(dicionario_overtime)
    df_limpo['Cargo_Traduzido'] = df_limpo['JobRole'].map(dicionario_cargos)
    
    # Convertendo colunas numéricas que podem ter sido lidas como texto
    colunas_numericas = [
        'Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
        'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
        'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating',
        'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
        'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
        'YearsSinceLastPromotion', 'YearsWithCurrManager'
    ]
    for col in colunas_numericas:
        df_limpo[col] = pd.to_numeric(df_limpo[col])

    print("✅ Dados limpos e preparados com sucesso!")
    print("Abaixo, uma amostra do DataFrame preparado:")
    display(df_limpo.head())

except Exception as e:
    print(f"\n❌ Ocorreu um erro na limpeza dos dados: {e}")


Passo 2: Aplicando limpeza e traduções...
✅ Dados limpos e preparados com sucesso!
Abaixo, uma amostra do DataFrame preparado:


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Situacao,HorasExtras_PT,Cargo_Traduzido
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,2,...,8,0,1,6,4,0,5,Saiu,Sim,Executivo(a) de Vendas
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,2,3,...,10,3,3,10,7,1,7,Ficou,Não,Cientista de Pesquisa
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,4,...,7,3,3,0,0,0,0,Saiu,Sim,Técnico(a) de Laboratório
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,5,4,...,8,3,3,8,7,3,0,Ficou,Sim,Cientista de Pesquisa
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,7,1,...,6,3,3,2,2,2,2,Ficou,Não,Técnico(a) de Laboratório


In [6]:
# --- PASSOS 3 E 4: CÁLCULO DO RISCO E SALVAMENTO FINAL ---

print("\nPasso 3: Calculando o Score de Risco para cada colaborador...")

try:
    # --- PASSO 3 ---
    def calcular_score_risco(funcionario):
        score = 0
        if funcionario['OverTime'] == 'Yes': score += 30
        if funcionario['JobSatisfaction'] <= 2: score += 20
        if funcionario['YearsSinceLastPromotion'] > 4: score += 15
        if funcionario['JobRole'] in ['Sales Representative', 'Laboratory Technician']: score += 15
        if funcionario['MonthlyIncome'] < 3500: score += 10
        if funcionario['YearsWithCurrManager'] == 0: score += 10
        return score

    df_limpo['Score_de_Risco'] = df_limpo.apply(calcular_score_risco, axis=1)

    def categorizar_risco(score):
        if score >= 50: return 'Alto Risco'
        elif score >= 25: return 'Risco Moderado'
        else: return 'Baixo Risco'

    df_limpo['Nivel_de_Risco'] = df_limpo['Score_de_Risco'].apply(categorizar_risco)
    print("✅ Score de Risco calculado com sucesso!")

    # --- PASSO 4 ---
    print("\nPasso 4: Preparando e salvando a tabela de resultados...")

    colunas_finais = [
        'EmployeeNumber', 'Cargo_Traduzido', 'MonthlyIncome', 'Score_de_Risco',
        'Nivel_de_Risco', 'HorasExtras_PT', 'JobSatisfaction', 'YearsSinceLastPromotion'
    ]
    df_resultados = df_limpo[colunas_finais]
    df_resultados_ordenado = df_resultados.sort_values(by='Score_de_Risco', ascending=False)

    try:
        aba_resultados = planilha.worksheet("Resultados_Analisados")
    except gspread.WorksheetNotFound:
        aba_resultados = planilha.add_worksheet(title="Resultados_Analisados", rows="1500", cols=len(colunas_finais)+1)

    from gspread_dataframe import set_with_dataframe
    aba_resultados.clear()
    set_with_dataframe(aba_resultados, df_resultados_ordenado)

    print("\n✅ SUCESSO! Pipeline concluído.")
    print("Os resultados foram salvos na aba 'Resultados_Analisados' da sua Planilha Google.")

except Exception as e:
    print(f"\n❌ Ocorreu um erro na etapa final: {e}")


Passo 3: Calculando o Score de Risco para cada colaborador...
✅ Score de Risco calculado com sucesso!

Passo 4: Preparando e salvando a tabela de resultados...

✅ SUCESSO! Pipeline concluído.
Os resultados foram salvos na aba 'Resultados_Analisados' da sua Planilha Google.


In [ ]:
!pip freeze > requirements.txt